In [13]:
import pandas as pd
import os
import matplotlib.pyplot as plt 
from sklearn.metrics import get_scorer, make_scorer, precision_score, recall_score, f1_score, f1_score, confusion_matrix, average_precision_score

def fnr(y_test, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    fnr = fn / (fn + tp)
    return fnr

scoring = {
    'accuracy': 'accuracy',
    'precision': make_scorer(precision_score, pos_label=1),
    'recall': make_scorer(recall_score, pos_label=1),
    'pr_auc': make_scorer(average_precision_score),
    'fnr': make_scorer(fnr),
    'f1_weighted': make_scorer(f1_score, average='weighted', pos_label=1)
}

train_metrics = pd.read_csv('logreg_results/logreg_metrics_train_results.csv')
test_metrics = pd.read_csv('logreg_results/logreg_metrics_test_results.csv')

merged = pd.merge(train_metrics, test_metrics, on='Antibiotic', suffixes=('_train', '_test'))

for metric_name, scorer_key in scoring.items():
    scorer = get_scorer(scorer_key)
    plt.figure(figsize=(14, 6))
    plt.bar(merged['Antibiotic'], merged[f'{metric_name}'], label=f'Training {metric_name}', alpha=0.2)
    plt.bar(merged['Antibiotic'], merged[f'{metric_name}_pre'], label=f'Testing {metric_name}', alpha=0.2, color='red')
    plt.bar(merged['Antibiotic'], merged[f'{metric_name}_post'], label=f'Calibration {metric_name}', alpha=0.2, color='orange')
    plt.tick_params(axis='x', rotation=45)
    plt.ylim(0.0, 0.9)
    plt.xlabel('Antibiotics', fontsize=16)
    plt.ylabel(f'{metric_name}', fontsize=16)
    plt.legend(loc='upper left')
    plt.tight_layout()
    output_dir = 'logreg_results/'
    plt.savefig(os.path.join(output_dir, f"logreg_train_vs_test_{metric_name}.png"))
    plt.close()

